<a href="https://colab.research.google.com/github/shandarmodabbir/FineTuning-BART-for-text-summarization/blob/main/BART_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers peft datasets
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq
import logging


In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension
!pip install jupyterlab-widgets

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Installing /usr/local/lib/python3.10/dist-packages/widgetsnbextension/static -> jupyter-js-widgets
Up to date: /usr/local/share/jupyter/nbextensions/jupyter-js-widgets/extension.js.LICENSE.txt
Up to date: /usr/local/share/jupyter/nbextensions/jupyter-js-widgets/extension.js
Up to date: /usr/local/share/jupyter/nbextensions/jupyter-js-widgets/extension.js.map
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable widgetsnbextension --py
    


In [ ]:

# Enable logging for progress tracking
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the dataset from Hugging Face Hub
arxiv_dataset = load_dataset("ccdv/arxiv-summarization")

# Use only 10% of the dataset
def sample_dataset(dataset, fraction=0.1):
    return dataset.shuffle(seed=42).select(range(int(len(dataset) * fraction)))

arxiv_dataset["train"] = sample_dataset(arxiv_dataset["train"], 0.1)
arxiv_dataset["validation"] = sample_dataset(arxiv_dataset["validation"], 0.1)


README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00003-of-00015.parquet:   0%|          | 0.00/227M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:

# Load the BART model and tokenizer
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Apply LoRA configuration
lora_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, lora_config)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["abstract"]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:

# Apply preprocessing with multiprocessing
def tokenize_and_save(dataset, name):
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names,
        num_proc=4  # Adjust based on available CPU cores
    )
    tokenized_dataset.save_to_disk(f"./tokenized_{name}_dataset")
    return tokenized_dataset


In [ ]:

train_dataset = tokenize_and_save(arxiv_dataset["train"], "train")
val_dataset = tokenize_and_save(arxiv_dataset["validation"], "val")


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/20303 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Saving the dataset (0/1 shards):   0%|          | 0/20303 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/643 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Saving the dataset (0/1 shards):   0%|          | 0/643 [00:00<?, ? examples/s]

In [ ]:

# Define data collator for padding during batching
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dacea472065ef26ad52a56c5daa369d0653e7cf9b035fc951b1c30f8e3168381
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

# Load the ROUGE metric using the evaluate library
rouge_metric = evaluate.load("rouge")

# Function to compute metrics
def compute_metrics(pred):
    # Decode predictions and references
    decoded_preds = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Extract the desired scores (e.g., ROUGE-2 F1 score)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result



In [ ]:

# Define training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-arxiv-summarization",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    gradient_accumulation_steps=4,
    dataloader_num_workers=4,
    fp16=torch.cuda.is_available(),
    report_to=["tensorboard"]
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Move model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()



In [ ]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
from transformers import TrainerCallback

class ProgressCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        # Called at the start of training.
        print("Training has started.")

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Logs training progress during the training process.
        if logs is not None:
            print(f"Step {state.global_step}: {logs}")

    def on_train_end(self, args, state, control, **kwargs):
        # Called at the end of training.
        print("Training has ended.")


# Add the custom callback
trainer.add_callback(ProgressCallback())


In [ ]:
# Start training
print("Training...")
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.save_model("./fine_tuned_bart")

# Save the tokenizer
tokenizer.save_pretrained("./fine_tuned_bart")

('./fine_tuned_bart/tokenizer_config.json',
 './fine_tuned_bart/special_tokens_map.json',
 './fine_tuned_bart/vocab.json',
 './fine_tuned_bart/merges.txt',
 './fine_tuned_bart/added_tokens.json')

In [ ]:
# Load the fine-tuned model and tokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

fine_tuned_model_path = "./fine_tuned_bart"
model = BartForConditionalGeneration.from_pretrained(fine_tuned_model_path)
tokenizer = BartTokenizer.from_pretrained(fine_tuned_model_path)

# Function to summarize a paragraph
def summarize_text(paragraph):
    inputs = tokenizer(paragraph, max_length=10240, return_tensors="pt", truncation=True)
    if torch.cuda.is_available():
        inputs = {key: value.cuda() for key, value in inputs.items()}
        model.cuda()
    summary_ids = model.generate(inputs["input_ids"], max_length=1280, min_length=30, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
paragraph = """Application of machine learning has become prominent in many fields and has captured
the imaginations of various industries. The development of data driven algorithms and the
ongoing digitization of subsurface geological measurements provide a world of
opportunities to maximize the exploration and production of resources such as oil, gas,
coal and geothermal energy. The current proliferation of data, democratization of state-of-
the-art processing technology and computation power provide an avenue for both large
and small industry players to maximize the use of their data to run more economic and
efficient operations. The aim of this thesis is to discuss the development of robust data-
driven methods and their effectiveness in providing insightful information about
subsurface properties. The study opens with a brief overview of the current literature
regarding application of data driven methods in the oil and gas industry.
Outlier detection can be a strenuous task when data preprocessing for purposes of
data- driven modeling. The thesis presents the efficacy of unsupervised outlier detection
algorithms for various practical cases by comparing the performance of four outlier
detection algorithms using appropriate metrics. Three case were created simulating: noisy
measurements, measurements from washout formation and measurements from
formations with several thin shale layers. It was observed that the Isolation Forest based
model is efficient in detecting a wide range of outlier types with a balanced accuracy score
of 0.88, 0.93 and 0.96 for the respective cases, while the DBSCAN based model was
effective at detecting outliers due to noisy measurement with balanced accuracy score of NMR measurements provide a wealth of geological information for petrophysical analysis
and can be key in accurately characterizing a reservoir, however they are expensive and
technically challenging to deploy, it has been shown in research that machine learning
models can be effective in synthesizing some log data. However, predicting an NMR
distribution where each depth is represented by several bins poses a different challenge.
In this study, a Random Forest model was used for predicting the NMR T1 distribution in
a well using relatively inexpensive and readily available well logs with an r2 score and
corrected Mean absolute percentage error of 0.14 and 0.84. The predictions fall within the
margin of error and an index was proposed to evaluate the reliability of each prediction
based on a quantile regression forest to provide the user more information on the accuracy
of the prediction when no data is available to test the model as will be the case in real
world application. Using this method engineers and geologist can obtain NMR derived
information from a well when no NMR tool has been run with a measure of reliability for
each predicted sample/depth."""

summary = summarize_text(paragraph)
print("Original Paragraph:")
print(paragraph)
print("\nGenerated Summary:")
print(summary)